In [2]:
import numpy as np
import pandas as pd
import json 
df = pd.read_csv("attribution_allocation_student_data.csv")
df = df.fillna('none')
df.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,email,referral,referral,none,none,1
1,True,referral,referral,email,none,none,1
2,True,referral,none,none,none,none,1
3,True,email,none,none,none,none,1
4,True,referral,referral,social,referral,none,1


In [9]:
df_channel = pd.read_csv("channel_spend_student_data.csv")
df_channel.head()

,Unnamed: 0,0
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [10]:
newdf = []
for i in range(len(df_channel)):
    temp = json.loads(df_channel['0'][i].replace("\'", "\""))
    temp['tier'] = (i+1)
    newdf.append(temp)
newdf

[{'direct': 0.0,
  'display': 1000.0,
  'email': 1000.0,
  'organic_search': 0.0,
  'paid_search': 999.9999999999999,
  'referral': 1000.0,
  'social': 1000.0000000000001,
  'tier': 1},
 {'direct': 0.0,
  'display': 2000.0,
  'email': 2000.0,
  'organic_search': 0.0,
  'paid_search': 2000.0,
  'referral': 2000.0,
  'social': 2000.0,
  'tier': 2},
 {'direct': 0.0,
  'display': 3000.0,
  'email': 3000.0,
  'organic_search': 0.0,
  'paid_search': 3000.0,
  'referral': 3000.0,
  'social': 3000.0,
  'tier': 3}]

In [20]:
newdf=pd.DataFrame(newdf)
newdf.head()

,direct,display,email,organic_search,paid_search,referral,social,tier
0,0.0,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1
1,0.0,2000.0,2000.0,0.0,2000.0,2000.0,2000.0,2
2,0.0,3000.0,3000.0,0.0,3000.0,3000.0,3000.0,3


# Method1 Last interaction

In [57]:
l=[]
for i in range(0,len(df)):
    info=df.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l.append([None,[0]])
                        
                        else:
                            l.append([[info['touch1']],[1]])    
                    else:
                        l.append([[info['touch2'],info['touch1']],[0,1]])    
                else:
                    l.append([[info['touch3'],info['touch2'],info['touch1']],[0,0,1]])    
            else:
                l.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0,0,0,1]])    
        else:
            l.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0,0,0,0,1]])
            


In [58]:
l

[[['referral', 'referral', 'email'], [0, 0, 1]],
 [['email', 'referral', 'referral'], [0, 0, 1]],
 [['referral'], [1]],
 [['email'], [1]],
 [['referral', 'social', 'referral', 'referral'], [0, 0, 0, 1]],
 [['email', 'referral', 'referral'], [0, 0, 1]],
 [['email'], [1]],
 [['email', 'referral', 'referral', 'display', 'referral'], [0, 0, 0, 0, 1]],
 [['email', 'referral', 'referral'], [0, 0, 1]],
 [['social', 'referral'], [0, 1]],
 [['email', 'referral', 'paid_search'], [0, 0, 1]],
 [['email', 'referral'], [0, 1]],
 [['referral'], [1]],
 [['referral', 'referral'], [0, 1]],
 [['email', 'referral', 'referral', 'referral'], [0, 0, 0, 1]],
 [['email', 'referral'], [0, 1]],
 [['email', 'referral', 'display', 'email'], [0, 0, 0, 1]],
 [['referral', 'email', 'referral', 'email'], [0, 0, 0, 1]],
 [['referral', 'referral', 'email'], [0, 0, 1]],
 [['email', 'referral'], [0, 1]],
 [['email'], [1]],
 [['referral', 'social', 'email'], [0, 0, 1]],
 [['email', 'direct'], [0, 1]],
 [['email', 'referral

In [59]:
len(l)

13779

In [60]:
d={}
for i in l:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d.keys():
            d[channel] = i[1][j]
        else:
            d[channel] += i[1][j]       
                
d

{'direct': 11,
 'display': 2059,
 'email': 1265,
 'organic_search': 10,
 'paid_search': 697,
 'referral': 7480,
 'social': 2257}

In [61]:
CAC={}
for i in d.keys():
    CAC[i] = (newdf[i].sum()/d[i])
    print(i,'CAC',round(CAC[i],2) ,sep = ' ')

referral CAC 0.8
email CAC 4.74
social CAC 2.66
display CAC 2.91
paid_search CAC 8.61
direct CAC 0.0
organic_search CAC 0.0


# Method2 Linear Model

In [62]:
l=[]
for i in range(0,len(df)):
    info=df.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l.append([None,[0]])
                        
                        else:
                            l.append([[info['touch1']],[1]])    
                    else:
                        l.append([[info['touch2'],info['touch1']],[0.5,0.5]])    
                else:
                    l.append([[info['touch3'],info['touch2'],info['touch1']],[1/3,1/3,1/3]])    
            else:
                l.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.25,0.25,0.25,0.25]])    
        else:
            l.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.2,0.2,0.2,0.2,0.2]])
            

In [63]:
l

[[['referral', 'referral', 'email'],
  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]],
 [['email', 'referral', 'referral'],
  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]],
 [['referral'], [1]],
 [['email'], [1]],
 [['referral', 'social', 'referral', 'referral'], [0.25, 0.25, 0.25, 0.25]],
 [['email', 'referral', 'referral'],
  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]],
 [['email'], [1]],
 [['email', 'referral', 'referral', 'display', 'referral'],
  [0.2, 0.2, 0.2, 0.2, 0.2]],
 [['email', 'referral', 'referral'],
  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]],
 [['social', 'referral'], [0.5, 0.5]],
 [['email', 'referral', 'paid_search'],
  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]],
 [['email', 'referral'], [0.5, 0.5]],
 [['referral'], [1]],
 [['referral', 'referral'], [0.5, 0.5]],
 [['email', 'referral', 'referral', 'referral'], [0.25, 0.25, 0.25, 0.25]],
 [['email', 'referral'], [0.5, 0.5]],
 [['em

In [64]:
d={}
for i in l:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d.keys():
            d[channel] = i[1][j]
        else:
            d[channel] += i[1][j]       
                
d

{'direct': 8.950000000000001,
 'display': 2031.3333333332862,
 'email': 1257.850000000008,
 'organic_search': 10.833333333333334,
 'paid_search': 707.983333333338,
 'referral': 7470.566666665257,
 'social': 2291.483333333311}

In [65]:
CAC={}
for i in d.keys():
    CAC[i] = (newdf[i].sum()/d[i])
    print(i,'CAC',round(CAC[i],2),sep = ' ')

referral CAC 0.8
email CAC 4.77
social CAC 2.62
display CAC 2.95
paid_search CAC 8.47
direct CAC 0.0
organic_search CAC 0.0


# Method3 Position Based

In [66]:
l=[]
for i in range(0,len(df)):
    info=df.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l.append([None,[0]])
                        
                        else:
                            l.append([[info['touch1']],[1]])    
                    else:
                        l.append([[info['touch2'],info['touch1']],[0.5,0.5]])    
                else:
                    l.append([[info['touch3'],info['touch2'],info['touch1']],[0.4,0.2,0.4]])    
            else:
                l.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,0.1,0.1,0.4]])    
        else:
            l.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,2/30,2/30,2/30,0.4]])
            

In [67]:
l

[[['referral', 'referral', 'email'], [0.4, 0.2, 0.4]],
 [['email', 'referral', 'referral'], [0.4, 0.2, 0.4]],
 [['referral'], [1]],
 [['email'], [1]],
 [['referral', 'social', 'referral', 'referral'], [0.4, 0.1, 0.1, 0.4]],
 [['email', 'referral', 'referral'], [0.4, 0.2, 0.4]],
 [['email'], [1]],
 [['email', 'referral', 'referral', 'display', 'referral'],
  [0.4, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.4]],
 [['email', 'referral', 'referral'], [0.4, 0.2, 0.4]],
 [['social', 'referral'], [0.5, 0.5]],
 [['email', 'referral', 'paid_search'], [0.4, 0.2, 0.4]],
 [['email', 'referral'], [0.5, 0.5]],
 [['referral'], [1]],
 [['referral', 'referral'], [0.5, 0.5]],
 [['email', 'referral', 'referral', 'referral'], [0.4, 0.1, 0.1, 0.4]],
 [['email', 'referral'], [0.5, 0.5]],
 [['email', 'referral', 'display', 'email'], [0.4, 0.1, 0.1, 0.4]],
 [['referral', 'email', 'referral', 'email'], [0.4, 0.1, 0.1, 0.4]],
 [['referral', 'referral', 'email'], [0.4, 0.2, 0.4]],
 [['email

In [68]:
d={}
for i in l:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d.keys():
            d[channel] = i[1][j]
        else:
            d[channel] += i[1][j]       
                
d

{'direct': 8.8,
 'display': 2113.5333333334306,
 'email': 1326.4333333333504,
 'organic_search': 10.4,
 'paid_search': 781.9333333333216,
 'referral': 7085.366666665451,
 'social': 2452.5333333334506}

In [69]:
CAC={}
for i in d.keys():
    CAC[i] = (newdf[i].sum()/d[i])
    print(i,'CAC',round(CAC[i],2),sep = ' ')

referral CAC 0.85
email CAC 4.52
social CAC 2.45
display CAC 2.84
paid_search CAC 7.67
direct CAC 0.0
organic_search CAC 0.0


# Marginal CAC with postion based

In [41]:
tier1=df.loc[df['tier'] == 1]
tier2=df.loc[df['tier'] == 2]
tier3=df.loc[df['tier'] == 3]

In [70]:
#cac for tier1
l1=[]
for i in range(0,len(tier1)):
    info=tier1.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l1.append([None,[0]])
                        
                        else:
                            l1.append([[info['touch1']],[1]])    
                    else:
                        l1.append([[info['touch2'],info['touch1']],[0.5,0.5]])    
                else:
                    l1.append([[info['touch3'],info['touch2'],info['touch1']],[0.4,0.2,0.4]])    
            else:
                l1.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,0.1,0.1,0.4]])    
        else:
            l1.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,2/30,2/30,2/30,0.4]])
d1={}
for i in l1:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d1.keys():
            d1[channel] = i[1][j]
        else:
            d1[channel] += i[1][j]       
                
CAC={}
for i in d1.keys():
    if i =='direct' or i=='organic_search':
        CAC[i] = (0/d1[i])
    else:
        CAC[i] = (1000/d1[i])
    print(i,'CAC',round(CAC[i],2),sep = ' ')

referral CAC 0.7
email CAC 3.87
social CAC 1.95
display CAC 2.05
paid_search CAC 7.07
direct CAC 0.0
organic_search CAC 0.0


In [51]:
len(l1)

2824

In [71]:
#cac for tier2
l2=[]
for i in range(0,len(tier2)):
    info=tier2.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l2.append([None,[0]])
                        
                        else:
                            l2.append([[info['touch1']],[1]])    
                    else:
                        l2.append([[info['touch2'],info['touch1']],[0.5,0.5]])    
                else:
                    l2.append([[info['touch3'],info['touch2'],info['touch1']],[0.4,0.2,0.4]])    
            else:
                l2.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,0.1,0.1,0.4]])    
        else:
            l2.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,2/30,2/30,2/30,0.4]])
d2={}
for i in l2:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d2.keys():
            d2[channel] = i[1][j]
        else:
            d2[channel] += i[1][j]       
                
CAC={}
for i in d2.keys():
    if i =='direct' or i=='organic_search':
        CAC[i] = (0/d2[i])
    else:
        CAC[i] = (1000/(d2[i]-d1[i]))
    print(i,'CAC',round(CAC[i],2),sep = ' ')

email CAC 5.18
referral CAC 0.9
display CAC 3.81
social CAC 2.71
paid_search CAC 7.75
organic_search CAC 0.0
direct CAC 0.0


In [53]:
len(l2)

4892

In [77]:
#cac for tier3
l3=[]
for i in range(0,len(tier3)):
    info=tier3.iloc[i,:]
    if info['convert_TF']==True:
        if info['touch5']== 'none':
            if info['touch4']=='none':
                if info['touch3']=='none':
                    if info['touch2']=='none':
                        if info['touch1']=='none':
                            l3.append([None,[0]])
                        
                        else:
                            l3.append([[info['touch1']],[1]])    
                    else:
                        l3.append([[info['touch2'],info['touch1']],[0.5,0.5]])    
                else:
                    l3.append([[info['touch3'],info['touch2'],info['touch1']],[0.4,0.2,0.4]])    
            else:
                l3.append([[info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,0.1,0.1,0.4]])    
        else:
            l3.append([[info['touch5'],info['touch4'],info['touch3'],info['touch2'],info['touch1']],[0.4,2/30,2/30,2/30,0.4]])
d3={}
for i in l3:
    for j in range(0,len(i[0])):
        channel = i[0][j]
        if channel not in d3.keys():
            d3[channel] = i[1][j]
        else:
            d3[channel] += i[1][j]       
                
CAC={}
for i in d3.keys():
    if i =='direct' or i=='organic_search':
        CAC[i] = (0/d3[i])
    else:
        CAC[i] = (1000/(d3[i]-d2[i]))
    print(i,'CAC',round(CAC[i],2) ,sep = ' ')

email CAC 6.07
referral CAC 1.67
paid_search CAC 10.07
display CAC 7.94
social CAC 5.64
direct CAC 0.0
organic_search CAC 0.0


In [49]:
len(l3)

6063